# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission, if necessary. Sections that begin with **'Implementation'** in the header indicate where you should begin your implementation for your project. Note that some sections of implementation are optional, and will be marked with **'Optional'** in the header.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [ ]:
# Load pickled data
import pickle

# TODO: Fill this in based on where you saved the training and testing data

training_file = "data/train.p"
testing_file = "data/test.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train_input, y_train_input = train['features'], train['labels']
X_test_input, y_test_input = test['features'], test['labels']

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 2D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below.

In [ ]:
### Replace each question mark with the appropriate value.

# TODO: Number of training examples
n_train = len(X_train_input)

# TODO: Number of testing examples.
n_test = len(X_test_input)

# TODO: What's the shape of an traffic sign image?
image_shape = [32,32]

# TODO: How many unique classes/labels there are in the dataset.
n_classes = 43

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc.

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections.

In [ ]:
### Data exploration visualization goes here.
### Feel free to use as many code cells as needed.
import matplotlib.pyplot as plt
import random
import csv
import numpy as np
# Visualizations will be shown in the notebook.
%matplotlib inline

def read_label_names(filename):
    with open(filename, newline='', encoding="utf8") as csvfile:
        spamreader = csv.reader(csvfile)
        vals = [x[1] for x in spamreader][1:]
        return vals

label_names = read_label_names('signnames.csv')

def show_samples(n_samples, images, labels, randomize = False):
    # Show a random sign with its label
    for sample_n in range(n_samples):
#         n = 1
#         while y_train_input[n] != 21:
        if randomize:
            n = random.randint(0, len(images))
        else:
            n = sample_n
        ax = plt.subplot(np.ceil(n_samples/3),3,sample_n+1)
        ax.set_title(label_names[labels[n]], fontsize=10)
        ax.imshow(images[n])
        ax.set_yticklabels([])
        ax.set_xticklabels([])
    plt.tight_layout()
    plt.show()
    

def print_train_labels_count():
    histogram = np.histogram(y_train_input, bins=np.arange(43))
    for i in range(len(histogram[0])):
        print("%s: %d" % (label_names[histogram[1][i]], histogram[0][i]))

print_train_labels_count()
show_samples(12, X_train_input, y_train_input, randomize = True)


----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

There are various aspects to consider when thinking about this problem:

- Neural network architecture
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

**NOTE:** The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play!

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:
# Normalize input data
import numpy as np
import cv2

normalize = True
sharpen = True
equalize = False
clahe = False

# Nothing to do to the labels
y_train = y_train_input
y_test = y_test_input

def equalize_histogram(img):
    img_hist_equalized = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    channels = cv2.split(img_hist_equalized)
    cv2.equalizeHist(channels[0], channels[0])
    cv2.merge(channels,img_hist_equalized)
    img_hist_equalized = cv2.cvtColor(img_hist_equalized, cv2.COLOR_YCrCb2RGB)
    return img_hist_equalized
    
X_train = X_train_input
X_test = X_test_input

def preprocess_image(img):
    if equalize:
        img = equalize_histogram(img)
    if clahe:
        cl = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(2,2))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2Lab);
        channels = cv2.split(img)
        # apply to the 'L' channel
        channels[0] = cl.apply(channels[0])
        img = cv2.merge(channels)
        img = cv2.cvtColor(img, cv2.COLOR_Lab2RGB);
    if normalize:
        img = (img - [128])/128
    if sharpen:
        kernel = np.array([[-1,-1,-1], [-1,10,-1], [-1,-1,-1]])
        img = cv2.filter2D(img, -1, kernel)
    return img

show_samples(12, X_test, y_test)

X_train = np.array([preprocess_image(im) for im in X_train ])
X_test =  np.array([preprocess_image(im) for im in X_test ])
        
show_samples(12, X_test, y_test)

### Question 1 

_Describe how you preprocessed the data. Why did you choose that technique?_

**Answer:**

In [ ]:
### Generate data additional data (OPTIONAL!)
### and split the data into training/validation/testing sets here.
### Feel free to use as many code cells as needed.
from sklearn.model_selection import train_test_split

# Separate a part of the test set for validation/ Don't run twice!!!
X_test, X_validation, y_test, y_validation = train_test_split(X_test, y_test, test_size=0.2)
print("Test size:", len(X_test), " labels size:", len(y_test) )
print("Validation size:", len(X_validation), " labels size:", len(y_validation) )

### Question 2

_Describe how you set up the training, validation and testing data for your model. **Optional**: If you generated additional data, how did you generate the data? Why did you generate the data? What are the differences in the new dataset (with generated data) from the original dataset?_

**Answer:**

In [ ]:
### Define your architecture here.
### Feel free to use as many code cells as needed.
import tensorflow as tf
from tensorflow.contrib.layers import flatten

def LeNet(x, weights, biases, keep_prob, keep_prob_conv):    
    # Hyperparameters
    mu = 0
    sigma = 0.1
    
    layer = 0
    
    conv1 = tf.nn.conv2d(x, weights[layer], [1,1,1,1], padding='SAME') + biases[layer]
    activation_conv1 = tf.nn.relu(conv1)
    layer += 1

    conv2 = tf.nn.conv2d(activation_conv1, weights[layer], [1,1,1,1], padding='SAME') + biases[layer]
    activation_conv2 =  tf.nn.relu(conv2)
    layer += 1
        
    conv3 = tf.nn.conv2d(activation_conv2, weights[layer], [1,1,1,1], padding='SAME') + biases[layer]
    activation_conv3 =  tf.nn.relu(conv3)
    layer += 1

    conv4 = tf.nn.conv2d(activation_conv3, weights[layer], [1,1,1,1], padding='SAME') + biases[layer]
    activation_conv4 =  tf.nn.relu(conv4)
    layer += 1
    
    conv5 = tf.nn.conv2d(activation_conv4, weights[layer], [1,1,1,1], padding='SAME') + biases[layer]
    activation_conv5 =  tf.nn.relu(conv5)
    layer += 1

    conv6 = tf.nn.conv2d(activation_conv5, weights[layer], [1,1,1,1], padding='SAME') + biases[layer]
    activation_conv6 =  tf.nn.relu(conv6)
    layer += 1
    
    conv7 = tf.nn.conv2d(activation_conv6, weights[layer], [1,1,1,1], padding='SAME') + biases[layer]
    activation_conv7 =  tf.nn.relu(conv7)
    layer += 1
    
    print(activation_conv7.get_shape())
    pool1 = tf.nn.max_pool(activation_conv7, [1, 2,2, 1], [1,2,2,1], padding='VALID')
    print(pool1.get_shape())
    
    conv8 = tf.nn.conv2d(pool1, weights[layer], [1,1,1,1], padding='SAME') + biases[layer]
    activation_conv8 =  tf.nn.relu(conv8)
    layer += 1

    conv9 = tf.nn.conv2d(activation_conv8, weights[layer], [1,1,1,1], padding='SAME') + biases[layer]
    activation_conv9 =  tf.nn.relu(conv9)
    layer += 1
    
    conv10 = tf.nn.conv2d(activation_conv9, weights[layer], [1,1,1,1], padding='SAME') + biases[layer]
    activation_conv10 =  tf.nn.relu(conv10)
    layer += 1
    
    print(activation_conv10.get_shape())
    pool2 = tf.nn.max_pool(activation_conv10, [1, 2,2, 1], [1,2,2,1], padding='VALID')
    print(pool2.get_shape())
    
    flat = flatten(pool2)
    fc1 = tf.matmul(flat, weights[layer]) + biases[layer]
    activation_fc1 = tf.nn.dropout(tf.nn.relu(fc1), keep_prob)
    layer += 1
    
    fc2 = tf.matmul(activation_fc1, weights[layer]) + biases[layer]
    activation_fc2 = tf.nn.dropout(tf.nn.relu(fc2), keep_prob)
    layer += 1
    
    logits = tf.matmul(activation_fc2, weights[layer]) + biases[layer]
    
    return logits

In [ ]:
#Training pipeline
rate = 0.001
# Hyperparameters
mu = 0
sigma = 0.1
EPOCHS = 20
BATCH_SIZE = 100
beta = 0.01

x = tf.placeholder(tf.float32, (None, 32, 32, 3))
y = tf.placeholder(tf.int32, (None))
keep_prob = tf.placeholder(tf.float32)
keep_prob_conv = tf.placeholder(tf.float32)

weights = [
    tf.Variable(tf.truncated_normal((3,3,3,32), mean = mu, stddev = sigma)),
    tf.Variable(tf.truncated_normal((3,3,32,32), mean = mu, stddev = sigma)),
    tf.Variable(tf.truncated_normal((3,3,32,32), mean = mu, stddev = sigma)),
    tf.Variable(tf.truncated_normal((3,3,32,32), mean = mu, stddev = sigma)),
    tf.Variable(tf.truncated_normal((3,3,32,32), mean = mu, stddev = sigma)),
    tf.Variable(tf.truncated_normal((3,3,32,32), mean = mu, stddev = sigma)),
    tf.Variable(tf.truncated_normal((3,3,32,32), mean = mu, stddev = sigma)),
    tf.Variable(tf.truncated_normal((3,3,32,64), mean = mu, stddev = sigma)),
    tf.Variable(tf.truncated_normal((3,3,64,64), mean = mu, stddev = sigma)),
    tf.Variable(tf.truncated_normal((3,3,64,64), mean = mu, stddev = sigma)),
    tf.Variable(tf.truncated_normal((4096, 120), mean = mu, stddev = sigma)),
    tf.Variable(tf.truncated_normal((120, 84), mean = mu, stddev = sigma)),
    tf.Variable(tf.truncated_normal((84, n_classes), mean = mu, stddev = sigma))
]
biases = [
    tf.Variable(tf.zeros(32)),
    tf.Variable(tf.zeros(32)),
    tf.Variable(tf.zeros(32)),
    tf.Variable(tf.zeros(32)),
    tf.Variable(tf.zeros(32)),
    tf.Variable(tf.zeros(32)),
    tf.Variable(tf.zeros(32)),
    tf.Variable(tf.zeros(64)),
    tf.Variable(tf.zeros(64)),
    tf.Variable(tf.zeros(64)),
    tf.Variable(tf.zeros(120)),
    tf.Variable(tf.zeros(84)),
    tf.Variable(tf.zeros(n_classes))
]

one_hot_y = tf.one_hot(y, n_classes)

logits = LeNet(x, weights, biases, keep_prob, keep_prob_conv)

cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, one_hot_y)
loss_operation = tf.reduce_mean(cross_entropy)

regularizer = tf.reduce_sum([tf.nn.l2_loss(w) for w in weights])
loss = tf.reduce_mean(loss_operation + beta * regularizer)
  
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

In [ ]:
# Accuracy evaluation
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.0, keep_prob_conv: 1.0})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

### Question 3

_What does your final architecture look like? (Type of model, layers, sizes, connectivity, etc.)  For reference on how to build a deep neural network using TensorFlow, see [Deep Neural Network in TensorFlow
](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/b516a270-8600-4f93-a0a3-20dfeabe5da6/concepts/83a3a2a2-a9bd-4b7b-95b0-eb924ab14432) from the classroom._


**Answer:**

In [ ]:
from sklearn.utils import shuffle

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    valid_losses = []
    train_losses = []
    train_accuracies = []
    valid_accuracies = []
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            res = sess.run(training_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 0.5, keep_prob_conv: 0.8})
        
        #train_loss = sess.run(loss_operation, feed_dict={x: X_train, y: y_train, keep_prob: 1.0, keep_prob_conv: 1.0})
        #valid_loss = sess.run(loss_operation, feed_dict={x: X_validation, y: y_validation, keep_prob: 1.0, keep_prob_conv: 1.0})
        #valid_losses.append(valid_loss)
        #train_losses.append(train_loss)

        #training_accuracy = evaluate(X_train, y_train)
        validation_accuracy = evaluate(X_validation, y_validation)
        
        #train_accuracies.append(training_accuracy)
        #valid_accuracies.append(validation_accuracy)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
#         print("Test Accuracy = {:.3f}".format(training_accuracy))
#         print("Diff = {:.3f}".format(training_accuracy - validation_accuracy))
        print()
        
    plt.plot(valid_losses, label="validation")
#     plt.plot(train_losses, label="training")
    plt.legend()
    plt.title("loss")
    plt.show()
    
#     plt.plot(train_accuracies, label="training")
    plt.plot(valid_accuracies, label="validation")
#     plt.legend(loc=4)
#     plt.title("accuracy")
#     plt.show()
    
    saver.save(sess, 'lenet')
    print("Model saved")

In [ ]:
# Test accuracy
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))

    test_accuracy = evaluate(X_test, y_test)
    print("Test Accuracy = {:.3f}".format(test_accuracy))

### Question 4

_How did you train your model? (Type of optimizer, batch size, epochs, hyperparameters, etc.)_


**Answer:**

### Question 5


_What approach did you take in coming up with a solution to this problem? It may have been a process of trial and error, in which case, outline the steps you took to get to the final solution and why you chose those steps. Perhaps your solution involved an already well known implementation or architecture. In this case, discuss why you think this is suitable for the current problem._

**Answer:**

---

## Step 3: Test a Model on New Images

Take several pictures of traffic signs that you find on the web or around you (at least five), and run them through your classifier on your computer to produce example results. The classifier might not recognize some local signs but it could prove interesting nonetheless.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:
### Load the images and plot them here.

import csv
import os
import matplotlib.image as mpimg

path = 'my_data/'
def read_csv_file():
    with open(path+'my_data.csv', newline='', encoding="utf8") as csvfile:
        spamreader = csv.reader(csvfile)
        vals = [(x[0],x[1]) for x in spamreader]
        return vals

files_and_labels = read_csv_file()    
files = [x for (x,y) in files_and_labels]

extra_imgs_orig =  [cv2.imread(path+f)[:,:,::-1] for f in files]
extra_img_count = len(extra_imgs_orig)

X_extra = [preprocess_image(img) for img in extra_imgs_orig]
y_extra  = [int(y) for (x,y) in files_and_labels]

np.set_printoptions(precision=2)

show_samples(6, extra_imgs_orig, y_extra, randomize = False)


### Question 6

_Choose five candidate images of traffic signs and provide them in the report. Are there any particular qualities of the image(s) that might make classification difficult? It could be helpful to plot the images in the notebook._



**Answer:**  
I chose some of my images intentionally difficult to solve, like the second image above, where the '30' in the max speed is the only uncovered part. This one proves really hard for the network to classify. For some tests it did find the right class, but the result wasn't very confident, which is expected because there are features in the image which are missing.  
I also found the third image shown was hard for the network to classify, I think it is because the shape of the numbers is different to the ones in the training samples. It is often classified as an 80km/h max speed sign.
I was surprised that the antenna reticullate in the background of the 'yield' sign isn't in general too much trouble, maybe because we're using color as a feature and the antenna is mostly just black against the sky in this image.  
The 'turn right' sign is also interesting, because the network seems to confuse this sign with the 'keep right' sign often. Since the arrows shape is the same and some pictures might have a slight rotation, both classes have very similar features and get classified closely.

In [ ]:
### Run the predictions here.
### Feel free to use as many code cells as needed.
np.set_printoptions(precision=2)

import csv
import os
import matplotlib.image as mpimg

# Define some operations
predicted = tf.argmax(logits, 1)
softmax_out = tf.nn.softmax(logits)

correct = 0
predictions = []
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    res = sess.run([softmax_out, predicted], feed_dict = {x: X_extra,keep_prob:1})
    
    for (prediction, processed_image, softmax_probs, label, j) in zip(res[1], X_extra, res[0], y_extra, range(extra_img_count)):
        top_k = sess.run(tf.nn.top_k(softmax_probs, k=5))
        
        if prediction == label:
            correct += 1
            
        predictions.append((prediction, label, top_k))
        
print("Accuracy = {:.3f}".format(correct/extra_img_count))


### Question 7

_Is your model able to perform equally well on captured pictures when compared to testing on the dataset? The simplest way to do this check the accuracy of the predictions. For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate._

_**NOTE:** You could check the accuracy manually by using `signnames.csv` (same directory). This file has a mapping from the class id (0-42) to the corresponding sign name. So, you could take the class id the model outputs, lookup the name in `signnames.csv` and see if it matches the sign from the image._


**Answer:**

In [ ]:
### Visualize the softmax probabilities here.
### Feel free to use as many code cells as needed.

i=0
f, axs = plt.subplots(3,3,figsize=(15,15))

# TODO SHOW WHETHER THE PREDICTION IS BETWEEN THE TOP-5 / show labels
for (prediction, label, top_l) in predictions:
    ax = plt.subplot(np.ceil(len(predictions)/3),3, i + 1)
    if prediction == label:
        ax.set_title("Accurate prediction", fontsize=10)
    plt.bar(range(5), top_l[0], 1)
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    i += 1
plt.tight_layout()
plt.show()



### Question 8

*Use the model's softmax probabilities to visualize the **certainty** of its predictions, [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. Which predictions is the model certain of? Uncertain? If the model was incorrect in its initial prediction, does the correct prediction appear in the top k? (k should be 5 at most)*

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

**Answer:**

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.